In [3]:
import utils.load_data as load_data
from collections import Counter
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from models.emotion_recognition import ClassifyEmotion
%matplotlib inline

In [4]:
%load_ext autoreload
%autoreload 2

In [43]:
# data, labels, metadata = load_data.load_sentences()

In [1]:
# lstm = ClassifyEmotion()
# lstm.train(data, labels, batch_size=64, epochs=1)

In [ ]:
train_data, train_labels, dev_data, dev_labels = load_data.load_common_voice_data(use_fbank=True) 